In [1]:
import pandas as pd

In [2]:
from autogluon import TabularPrediction as task

In [3]:
data= pd.read_csv('../Data/indicator_dataset/indicator_dataset.csv')

In [4]:
def drop_prefix(self, prefix):
    self.columns = self.columns.str.lstrip(prefix)
    return self

pd.core.frame.DataFrame.drop_prefix = drop_prefix

In [5]:
def change_set(last_states, data):
    current_data = data
    new_data = data
    for i in range(last_states):
        new_data = new_data.shift(1)
        if new_data.columns[0][0].isdigit():
            new_data = new_data.drop_prefix('{}_behind_'.format(i))
            new_data = new_data.add_prefix('{}_behind_'.format(i+1))
        else:
            new_data = new_data.add_prefix('{}_behind_'.format(i+1))
        
        current_data = pd.concat([current_data, new_data], axis = 1)
    return current_data

In [6]:
df = change_set(5, data)

In [7]:
train = df.iloc[:int(len(df) * 0.5), ]
test = df.iloc[int(len(df) * 0.5):, ]
del df

In [8]:
targets = pd.read_csv('../Data/ground_truth/ground_truth.csv')

In [ ]:
possible_coeffs = ['0.25', '0.5']
possible_types = []

In [9]:
label_type = 'label'
coeff = '0.25'

label_column = '{}_{}'.format(label_type, coeff)
targets = targets[label_column]
train_targets = targets[:int(len(targets) * 0.5)]
test_targets = targets[int(len(targets) * 0.5):]

In [10]:
train[label_column] = train_targets
test[label_column] = test_targets

In [ ]:
predictor = task.fit(train_data=train, label=label_column, presets='best_quality', eval_metric='average_precision')

No output_directory specified. Models will be saved in: AutogluonModels/ag-20201027_215715/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20201027_215715/
AutoGluon Version:  0.0.14
Train Data Rows:    251726
Train Data Columns: 1530
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    105049.58 MB
	Train Data (Original)  Memory Usage: 3081.13 MB (2.9% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify

In [27]:
prediction = predictor.predict_proba(test)
perf = predictor.evaluate_predictions(y_true=test_targets, y_pred=prediction, auxiliary_metrics=True)

Evaluation: average_precision on test data: 0.48955935837834247
Evaluations on test data:
{
    "average_precision": 0.48955935837834247
}


In [28]:
len(np.where(test_targets == 1)[0])/len(test_targets)

0.4610767262817508

In [29]:
coeff

'0.5'

In [30]:
predictor.output_directory

'AutogluonModels/ag-20201027_220401/'

In [31]:
import numpy as np
np.savetxt('updated_predictions_{}_{}.csv'.format(label_type, coeff), prediction, delimiter = ',')